# <u> Dataset Creation 

In [1]:
def retrieve_character_dialogues(character,file):
    dialogues = []
    with open(file) as f:
        for line in f:
            search_line = line.lower()
            anya_found = search_line.find(character)
            comma_found = search_line.find(',,')
            if anya_found != -1 and comma_found != -1 and anya_found < comma_found:
                dialogues.append(line[comma_found+2:])
    return dialogues

## after uploading all .ass files and folders to the current working directory

In [2]:
from pathlib import Path
import re

In [3]:
bracket_content = re.compile(r'\{.*?\}|shock!|\n')

In [4]:
p = Path('.')
paths = list(p.glob('**/*.eng.ass'))
character = 'anya'
dialogues = []
for path in paths:
    dialogues.extend(retrieve_character_dialogues(character,path))
cleaned_dialogues = bracket_content.sub("","".join(dialogues))

## Preliminary Checks

In [5]:
import sys

size_kb = sys.getsizeof(cleaned_dialogues) // 1024
print(f"Memory size: {size_kb} KB")

print(f"Number of dialogues: {len(cleaned_dialogues)}")

chars = sorted(set(cleaned_dialogues))
vocab_size = len(chars)
print(f"Vocabulary size: {vocab_size}")
print(f"Possible Vocabulary = [{''.join(chars)}]")


Memory size: 101 KB
Number of dialogues: 51963
Vocabulary size: 69
Possible Vocabulary = [ !"'(),-.012578:?ABCDEFGHIJKLMNOPRSTUVWYZ\abcdefghijklmnopqrstuvwxyz—]


> ### <u> NOTE: </u>
>
> - <b>The amount of data is quite low but we dont really care about overfitting for this model right now, although the we still  should have around an MB of 
> data if possible
>   
>
> <br>

## Encoder and Decoder Functions

In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s]
decode = lambda l : ''.join(itos[i] for i in l)

In [7]:
encoded_text = encode("Hehe")
print('encoded text : ', encoded_text)
decoded_text = decode(encoded_text)
print('decoded text :',decoded_text)

encoded text :  [24, 46, 49, 46]
decoded text : Hehe


## Storing data

In [8]:
import torch 
data = torch.tensor(encode(cleaned_dialogues),dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([51963]) torch.int64


In [9]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

## Splitting data

In [10]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data [n:]

## visualizing one training example 

In [11]:
blocksize = 8 
x = train_data[:blocksize]
y = train_data[1:blocksize+1]
for t in range(blocksize):
    context = x[:t+1]
    target = y[t]
    print(f'given context {context} the next predicted token should be {target}')

given context tensor([25]) the next predicted token should be 0
given context tensor([25,  0]) the next predicted token should be 42
given context tensor([25,  0, 42]) the next predicted token should be 54
given context tensor([25,  0, 42, 54]) the next predicted token should be 0
given context tensor([25,  0, 42, 54,  0]) the next predicted token should be 17
given context tensor([25,  0, 42, 54,  0, 17]) the next predicted token should be 55
given context tensor([25,  0, 42, 54,  0, 17, 55]) the next predicted token should be 66
given context tensor([25,  0, 42, 54,  0, 17, 55, 66]) the next predicted token should be 42


a sequence of n characters typically has n-1 training examples, we take (n+1) characters  to match the 'block_size' or number of training examples per sequence. eg: for block_size 8 we would consider a sequence of 9 characters so we get 8 training examples

## Creating Data Loader

In [12]:
#setting seed for consistent output
torch.manual_seed(74)
batch_size = 4 
block_size = 8 
# gets 4 sequences of 8 characters
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    return x,y


In [13]:
xb, yb = get_batch('train')

print("Inputs (xb):")
print(f"  Shape: {xb.shape}, \n Values: {xb}")
print("\nTargets (yb):")
print(f"  Shape: {yb.shape}, \n Values: {yb}")


Inputs (xb):
  Shape: torch.Size([4, 8]), 
 Values: tensor([[43, 46, 60, 61,  8, 30, 56, 64],
        [ 0, 17, 55, 66, 42,  0, 44, 42],
        [25,  0, 64, 42, 55, 61,  0, 61],
        [63, 46, 60,  1,  0, 23, 50, 63]])

Targets (yb):
  Shape: torch.Size([4, 8]), 
 Values: tensor([[46, 60, 61,  8, 30, 56, 64,  0],
        [17, 55, 66, 42,  0, 44, 42, 55],
        [ 0, 64, 42, 55, 61,  0, 61, 56],
        [46, 60,  1,  0, 23, 50, 63, 46]])


In [14]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]   # Show context slice up to t+1
        target = yb[b, t]       # Show current target
        print(f"Batch: {b}, Time Step: {t+1}")
        print(f"Context: {context}")
        print(f"Target: {target}")
        print("_" * 40)         


Batch: 0, Time Step: 1
Context: tensor([43])
Target: 46
________________________________________
Batch: 0, Time Step: 2
Context: tensor([43, 46])
Target: 60
________________________________________
Batch: 0, Time Step: 3
Context: tensor([43, 46, 60])
Target: 61
________________________________________
Batch: 0, Time Step: 4
Context: tensor([43, 46, 60, 61])
Target: 8
________________________________________
Batch: 0, Time Step: 5
Context: tensor([43, 46, 60, 61,  8])
Target: 30
________________________________________
Batch: 0, Time Step: 6
Context: tensor([43, 46, 60, 61,  8, 30])
Target: 56
________________________________________
Batch: 0, Time Step: 7
Context: tensor([43, 46, 60, 61,  8, 30, 56])
Target: 64
________________________________________
Batch: 0, Time Step: 8
Context: tensor([43, 46, 60, 61,  8, 30, 56, 64])
Target: 0
________________________________________
Batch: 1, Time Step: 1
Context: tensor([0])
Target: 17
________________________________________
Batch: 1, Time Ste

# SIMPLEST BASELINE

## Defining the Bigram

In [20]:
import torch 
import torch.nn as nn
from einops import rearrange
from torch.nn import functional as F

torch.manual_seed(74)
#TL:DR each row maps to a character and gives logits or it, i.e next character predictions given an input characters idx
class BigramLanguageModel(nn.Module):
    
    def __init__(self,vocab_size):
        super().__init__()
        # the trainable params here are just weights of size vocab_size * vocab_size
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            logits = rearrange(logits, 'b t c -> (b t) c')
            targets = rearrange(targets, 'b t -> (b t)' )
            loss = F.cross_entropy(logits,targets)
        
        return logits,loss
    def generate(self,idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(idx)

            logits = logits[:, -1 , :]
            
            probs = F.softmax(logits,dim=-1)
            idx_next =torch.multinomial(probs,num_samples=1)
            idx = torch.cat((idx,idx_next),dim=1)
        return idx
m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(loss)
print(decode(m.generate(idx = torch.zeros((1,1),dtype=torch.long), max_new_tokens=100)[0].tolist()))

tensor(4.6637, grad_fn=<NllLossBackward0>)
 t,Oi!2eAo D2)igt!R:K7OJiutoNybkO:—?Ob0Pv7aTVx"a)OluxG.kK—ojDLKe7H!E oa-fgLn0BA)mhN:KmFLvL2Fo0PrmEdLD


> **Note**  
> Even though we pass the entire sequence of logits into the next forward pass, this is still a **bigram model** — it only uses the **immediately previous character** to predict the next one.  
>  
> The reason we concatenate the entire sequence and then take only the last prediction is for **consistency**. The same forward function can then be reused for more complex models (like Transformers), where the full sequence context actually matters.
>
> also to use the model on gpu both the data and the parameters need to be offloaded to the gpu to work

In [16]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-2)

In [17]:
batch_size = 32
for step in range(10000):
    
    xb,yb = get_batch('train')
    
    logits , loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print("loss : ",loss.item())

loss :  2.4704837799072266


In [18]:
print(decode(m.generate(idx = torch.zeros((1,1),dtype=torch.long), max_new_tokens=50)[0].tolist()))


 wa, buss t's!Gis g gorrdicomeven't tes!Shoocuy's..
